In [16]:
import os
import glob
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.4.1'

In [3]:
os.listdir('dataset')

['single_prediction', 'test_set', 'training_set']

In [4]:
# Feature scaling is absolutely necessary for Neural Networks
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [5]:
# Initialising the CNN
cnn = tf.keras.models.Sequential()
# Convolution Layer - 1
cnn.add(tf.keras.layers.Conv2D(filters = 32, 
                               kernel_size = 3,
                               activation = 'relu',
                               input_shape = [64,64,3]))
# MaxPooling Layer - 1
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2,
                                  strides = 2))
# Convolution Layer - 2
cnn.add(tf.keras.layers.Conv2D(filters = 32,
                                kernel_size = 3,
                                activation = 'relu'))
# MaxPooling Layer - 2
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2,
                                  strides = 2))
# Flattening Layer
cnn.add(tf.keras.layers.Flatten())
# Full Connection Layer - 1
cnn.add(tf.keras.layers.Dense(units = 128,
                              activation = 'relu'))
# Full Connection Layer - 2
cnn.add(tf.keras.layers.Dense(units = 1,
                              activation = 'sigmoid'))
# Compiling the Model
cnn.compile(optimizer = 'adam',
            loss = 'binary_crossentropy',
            metrics = ['accuracy'])

In [7]:
cnn.fit(x = training_set, 
        validation_data = test_set,
        epochs = 25)

Epoch 1/25
250/250 [==============================] - 234s 915ms/step - loss: 0.6846 - accuracy: 0.5655 - val_loss: 0.6401 - val_accuracy: 0.6130
Epoch 2/25
250/250 [==============================] - 58s 234ms/step - loss: 0.6029 - accuracy: 0.6739 - val_loss: 0.5566 - val_accuracy: 0.7250
Epoch 3/25
250/250 [==============================] - 60s 240ms/step - loss: 0.5485 - accuracy: 0.7169 - val_loss: 0.5173 - val_accuracy: 0.7560
Epoch 4/25
250/250 [==============================] - 58s 232ms/step - loss: 0.5310 - accuracy: 0.7381 - val_loss: 0.5095 - val_accuracy: 0.7475
Epoch 5/25
250/250 [==============================] - 57s 230ms/step - loss: 0.5042 - accuracy: 0.7538 - val_loss: 0.5021 - val_accuracy: 0.7575
Epoch 6/25
250/250 [==============================] - 58s 231ms/step - loss: 0.4710 - accuracy: 0.7719 - val_loss: 0.5077 - val_accuracy: 0.7680
Epoch 7/25
250/250 [==============================] - 58s 232ms/step - loss: 0.4589 - accuracy: 0.7862 - val_loss: 0.4725 - val_a

In [8]:
print(cnn.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               802944    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

In [10]:
#Making the Prediction
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', 
                             target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'
print(prediction)

dog


In [11]:
#Making the Prediction
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', 
                             target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'
print(prediction)

cat


In [15]:
# Saving our Model
cnn.save("cats_or_dogs.h5")